# Multivariate Time Series Prediction Using LSTM

In this example, we will implement an LSTM using mlpack to forecast multivariate time series data.

## Linking libraries to the Notebook.

Link all required libraries to jupyter notebook.
I think this section should be removed, Let me know what you think.

In [1]:
#pragma cling add_library_path("/usr/local/lib/")
#pragma cling add_include_path("/usr/local/include/")
#pragma cling load("/usr/local/Cellar/armadillo/9.850.1_1/lib/libarmadillo.9.85.1.dylib")
#pragma cling load("/usr/local/Cellar/armadillo/9.850.1_1/lib/libarmadillo.9.dylib")
#pragma cling load("/usr/local/Cellar/armadillo/9.850.1_1/lib/libarmadillo.dylib")
#pragma cling load("/usr/local/Cellar/mlpack/3.2.2_2/lib/libmlpack.3.2.dylib")

## 1. Setup

Include all libraries required to implement this tutorial. These mainly include files from mlpack, ensmallen and armadillo.

In [2]:
#include <mlpack/core.hpp>
#include <mlpack/prereqs.hpp>
#include <mlpack/methods/ann/rnn.hpp>
#include <mlpack/methods/ann/layer/layer.hpp>
#include <mlpack/core/data/scaler_methods/min_max_scaler.hpp>
#include <mlpack/methods/ann/init_rules/he_init.hpp>
#include <mlpack/methods/ann/loss_functions/mean_squared_error.hpp>
#include <mlpack/core/data/split_data.hpp>
#include <ensmallen.hpp>
#include <armadillo>

Some convienent namespaces to simplify the tutorial.

In [3]:
using namespace mlpack;
using namespace mlpack::ann;
using namespace arma;
using namespace std;
using namespace ens;

## 2. Set Model and Training parameters.

Set the training parameters for the model

In [4]:
// If true, the model will be trained; if false, the saved model will be
// read and used for prediction
// NOTE: Training the model may take a long time, therefore once it is
// trained you can set this to false and use the model for prediction.
// NOTE: There is no error checking in this example to see if the trained
// model exists!
const bool bTrain = true;
// You can load and further train a model by setting this to true.
const bool bLoadAndTrain = false;

// Testing data is taken from the dataset in this ratio.
const double RATIO = 0.1;

// Step size of an optimizer.
const double STEP_SIZE = 5e-5;

// Number of cells in the LSTM (hidden layers in standard terms).
// NOTE: you may play with this variable in order to further optimize the
// model (as more cells are added, accuracy is likely to go up, but training
// time may take longer).
const int H1 = 25;

// Number of data points in each iteration of SGD.
const size_t BATCH_SIZE = 16;

// Nunmber of timesteps to look backward for in the RNN.
const int rho = 25;

// Max Rho for LSTM.
const int maxRho = rho;

// Number of epochs for training.
const int EPOCHS = 150;

Set paths for the dataset, trained model and final predictions.

In [5]:
// Change the names of these files as necessary. They should be correct
// already, if your program's working directory contains the data and/or
// model.
const std::string dataFile = "Google2016-2019.csv";
// Path where the model will be saved.
const std::string modelFile = "lstm_multi.bin";
// Path where the final predicitions will be stored.
const std::string predFile = "lstm_multi_predictions.csv";

## 2. Loading and Preprocess the Dataset.

Dataset will be loaded and preprocessed for the model. If we want to predict the Google stock price correctly then we need to consider the volume of the stocks traded, the closing, opening, high and low values of the stock price from the previous days. This is a time series problem.
We will create data for the training of the RNN model that will go back 25 business days in the past for each time step.
We will convert the input data to the time series format the RNN requires. We will take 30 % of the latest data as our test dataset.

In [6]:
arma::mat dataset;

// In Armadillo rows represent features, columns represent data points.
std::cout << "Reading data ..." << std::endl;
mlpack::data::Load(dataFile, dataset, true);

Reading data ...


In [7]:
// Visualize the first 1 rows of the dataset.
dataset.submat(0, 0, dataset.n_rows - 1, 3).print()
// Here each row represents: date, close, volume, open, high, low.

            0   2.7000e+01   2.8000e+01   2.9000e+01
            0   6.6826e+02   6.8004e+02   6.8411e+02
            0   2.6320e+06   2.1697e+06   1.9314e+06
            0   6.7100e+02   6.7897e+02   6.8300e+02
            0   6.7230e+02   6.8033e+02   6.8743e+02
            0   6.6328e+02   6.7300e+02   6.8141e+02


### Preprocess the dataset.

In [8]:
// The CSV file has a header, so it is necessary to remove it. In Armadillo's
// representation it is the first column.
// The first column in the CSV is the date which is not required, therefore
// we remove it also (first row in in arma::mat).

dataset = dataset.submat(1, 1, dataset.n_rows - 1, dataset.n_cols - 1);

// We have 5 input data columns and 2 output columns (target).
size_t inputSize = 5, outputSize = 2;

// Split the dataset into training and validation sets.
arma::mat trainData = dataset.submat(arma::span(),arma::span(0, (1 - RATIO) *
      dataset.n_cols));
arma::mat testData = dataset.submat(arma::span(), arma::span((1 - RATIO) * dataset.n_cols,
      dataset.n_cols - 1));

Data needs to be scaled before we train or test the model. This reduces variation in data (normalize the data) and makes the algorithm converge faster.

In [9]:
mlpack::data::MinMaxScaler scale;
// Fit scaler only on training data.
scale.Fit(trainData);
scale.Transform(trainData, trainData);
scale.Transform(testData, testData);

### Create Time Series Dataset.

The time series data for training the model contains the Closing stock price, the Volume of stocks traded,
Opening stock price, Highest stock price and Lowest stock price for 'rho' days in the past. 
The two target variables (multivariate) we want to predict are the Highest stock price and Lowest stock price
(high, low) for the next day! 

In [10]:
template<typename InputDataType = arma::mat,
         typename DataType = arma::cube,
         typename LabelType = arma::cube>
void CreateTimeSeriesData(InputDataType dataset,
                          DataType& X,
                          LabelType& y,
                          const size_t rho)
{
  for (size_t i = 0; i < dataset.n_cols - rho; i++)
  {
    X.subcube(arma::span(), arma::span(i), arma::span()) =
        dataset.submat(arma::span(), arma::span(i, i + rho - 1));
    y.subcube(arma::span(), arma::span(i), arma::span()) =
        dataset.submat(arma::span(3, 4), arma::span(i + 1, i + rho));
  }
}

In [11]:
// We need to individually create training and testing time series data.
arma::cube trainX, trainY, testX, testY;
trainX.set_size(inputSize, trainData.n_cols - rho + 1, rho);
trainY.set_size(outputSize, trainData.n_cols - rho + 1, rho);
testX.set_size(inputSize, testData.n_cols - rho + 1, rho);
testY.set_size(outputSize, testData.n_cols - rho + 1, rho);

// Create training sets for one-step-ahead regression.
CreateTimeSeriesData(trainData, trainX, trainY, rho);
// Create test sets for one-step-ahead regression.
CreateTimeSeriesData(testData, testX, testY, rho);

## 3. Create the Model

We add 3 LSTM modules that will be stacked one after the other in the RNN, implementing an efficient stacked RNN. Finally, the output will have 2 units the (high, low) values of the stock price for the next day.

In [12]:
mlpack::ann::RNN<mlpack::ann::MeanSquaredError<>, mlpack::ann::RandomInitialization> model(rho);

if (bLoadAndTrain)
{
  // The model will be trained further.
  std::cout << "Loading and further training model..." << std::endl;
  data::Load(modelFile, "LSTMMulti", model);
}
else
{
  model.Add<mlpack::ann::IdentityLayer<> >();
  model.Add<mlpack::ann::LSTM<> >(inputSize, H1, maxRho);
  model.Add<mlpack::ann::Dropout<> >(0.5);
  model.Add<mlpack::ann::LeakyReLU<> >();
  model.Add<mlpack::ann::LSTM<> >(H1, H1, maxRho);
  model.Add<mlpack::ann::Dropout<> >(0.5);
  model.Add<mlpack::ann::LeakyReLU<> >();
  model.Add<mlpack::ann::LSTM<> >(H1, H1, maxRho);
  model.Add<mlpack::ann::LeakyReLU<> >();
  model.Add<mlpack::ann::Linear<> >(H1, outputSize);
}

### 4.Training the model.

We will use ensmallen to get the optimizer and train the model. For more details refer to the [documentation](https://www.ensmallen.org/docs.html).

In [13]:
// Setting parameters Stochastic Gradient Descent (SGD) optimizer.
ens::SGD<ens::AdamUpdate> optimizer(
    STEP_SIZE, // Step size of the optimizer.
    BATCH_SIZE, // Batch size. Number of data points that are used in each iteration.
    trainData.n_cols * EPOCHS, // Max number of iterations.
    1e-8,// Tolerance.
    true, // Shuffle.
    ens::AdamUpdate(1e-8, 0.9, 0.999)); // Adam update policy.

optimizer.Tolerance() = -1;

In [14]:
// Train the model.
if (bTrain)
{
 std::cout << "Training ..." << std::endl;

 model.Train(trainX,
             trainY,
             optimizer,
             // PrintLoss Callback prints loss for each epoch.
             ens::PrintLoss(),
             // Progressbar Callback prints progress bar for each epoch.
             ens::ProgressBar(),
             // Stops the optimization process if the loss stops decreasing
             // or no improvement has been made. This will terminate the
             // optimization once we obtain a minima on training set.
             ens::EarlyStopAtMinLoss());
}

Training ...
Epoch 1/2433
nan42 [====================================================================================================] 100% - ETA: 0s - loss: nan2494
42/42 [====================================================================================================] 100% - 13s 328ms/step - loss: nan


In [15]:
// Save the model.
mlpack::data::Save(modelFile, "LSTMMulti", model);
std::cout << "Model saved in " << modelFile << std::endl;

Model saved in lstm_multi.bin


## 5. Load the Trained Model and Run Inference.

Load the the model with saved weights and run inference on testing data.

In [16]:
mlpack::ann::RNN<mlpack::ann::MeanSquaredError<>, mlpack::ann::RandomInitialization> trainedModel(rho);
std::cout << "Loading model ..." << std::endl;
mlpack::data::Load(modelFile, "LSTMMulti", trainedModel);

Loading model ...


In [17]:
// Run Inference.
arma::cube predOutP;
trainedModel.Predict(testX, predOutP);

### Calculate the validation loss.

For this purpose we will be calculating Mean Squared error.

In [18]:
/*
 * Function to calcute MSE for arma::cube.
 */
double MSE(arma::cube &pred, arma::cube &Y)
{
  return metric::SquaredEuclideanDistance::Evaluate(pred, Y) / (Y.n_elem);
}

In [19]:
// Calculate error on predicted data.
double testMSEP = MSE(predOutP, testY);
std::cout << "Mean Squared Error on Prediction data points:= " << testMSEP << std::endl;

Mean Squared Error on Prediction data points:= nan


## 6. Save the results.
Since the predicted is in form of a cube we need to convert it to matrix with predicted values. Then we take the inverse transform to convert it to meaningful data.

In [20]:
/**
 * This function saves the input data for prediction and the prediction results
 * in CSV format. The prediction results are the (high, low) for the next day
 * and come from the last slice of the prediction. The last 2 columns are the
 * predictions; the preceding columns are the data used to generate those
 * predictions.
 */
void SaveResults(const std::string filename,
                 const arma::cube& predictions,
                 data::MinMaxScaler& scale,
                 const arma::cube& testX)
{
  arma::mat flatDataAndPreds = testX.slice(testX.n_slices - 1);

  // The prediction results are the (high, low) for the next day and come from
  // the last slice from the prediction.
  flatDataAndPreds.rows(flatDataAndPreds.n_rows - 2,
                        flatDataAndPreds.n_rows - 1) = predictions.slice(predictions.n_slices - 1);

  scale.InverseTransform(flatDataAndPreds, flatDataAndPreds);

  // We need to remove the last column because it was not used for training
  // (there is no next day to predict).
  flatDataAndPreds.shed_col(flatDataAndPreds.n_cols - 1);

  // Save the data to file. The last columns are the predictions; the preceding
  // columns are the data used to generate those predictions.
  mlpack::data::Save(filename, flatDataAndPreds);

  // Print the output to screen.
  // NOTE: we do not have the last data point in the input for the prediction
  // because we did not use it for the training, therefore the prediction result
  // will be for the day before. In your own application you may of course load
  // any dataset for prediction.
  std::cout << "The predicted Google stock (high, low) for the last day is: " << std::endl;
  std::cout << "  (" << flatDataAndPreds(flatDataAndPreds.n_rows - 2, flatDataAndPreds.n_cols - 1) << ", ";
  std::cout << flatDataAndPreds(flatDataAndPreds.n_rows - 1, flatDataAndPreds.n_cols - 1) << ")" << std::endl;
}

In [21]:
// Save the output predictions and show the results.
SaveResults(predFile, predOutP, scale, testX);

The predicted Google stock (high, low) for the last day is: 
  (nan, nan)
